In [49]:
import pandas as pd #Data manipulation
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:
#crete a new dataset with 2 columns - Genre and Description
df = pd.read_csv('/content/films.txt', delimiter=":::", engine='python', names=["ID", "Title", "Genre", "Description"])
df = df.drop(['ID', 'Title'], axis = 1)
df

,Genre,Description
0,drama,Listening in to a conversation between his do...
1,thriller,A brother and sister with a past incestuous r...
2,adult,As the bus empties the students for their fie...
3,drama,To help their unemployed father make ends mee...
4,drama,The film's title refers not only to the un-re...
...,...,...
54209,comedy,This short-lived NBC live sitcom centered on ...
54210,horror,The NEXT Generation of EXPLOITATION. The sist...
54211,documentary,"Ze bestaan echt, is a stand-up comedy about g..."
54212,comedy,Walter and Vivian live in the country and hav...


In [42]:
df['Genre'].value_counts()
#all is good, we have diffrent naming for categories. there is no need to worry.


,count
Genre,
drama,13613
documentary,13096
comedy,7447
short,5073
horror,2204
thriller,1591
action,1315
western,1032
reality-tv,884


In [43]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Genre_id'] = label_encoder.fit_transform(df['Genre'])
df

,Genre,Description,Genre_id
0,drama,Listening in to a conversation between his do...,8
1,thriller,A brother and sister with a past incestuous r...,24
2,adult,As the bus empties the students for their fie...,1
3,drama,To help their unemployed father make ends mee...,8
4,drama,The film's title refers not only to the un-re...,8
...,...,...,...
54209,comedy,This short-lived NBC live sitcom centered on ...,5
54210,horror,The NEXT Generation of EXPLOITATION. The sist...,13
54211,documentary,"Ze bestaan echt, is a stand-up comedy about g...",7
54212,comedy,Walter and Vivian live in the country and hav...,5


In [53]:
tfidf = TfidfVectorizer(sublinear_tf=True,      # Use sublinear scaling to reduce the impact of high term frequency
                        min_df=5,              # Ignore terms that appear in fewer than 5 documents
                        ngram_range=(1, 2),    # Consider both unigrams (single words) and bigrams (two consecutive words)
                        stop_words='english')  # Remove English stop words

# Transform the movie descriptions into a numerical feature matrix
features = tfidf.fit_transform(df['Description'])
labels = df['Genre_id']
# Output the number of descriptions and features in the TF-IDF matrix
print("Each of the %d movie descriptions is represented by %d features (TF-IDF score of unigrams and bigrams)" % features.shape)


Each of the 54214 movie descriptions is represented by 70972 features (TF-IDF score of unigrams and bigrams)


In [54]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)
df['Description_clean'] = df['Description'].apply(preprocess_text)

# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(
    sublinear_tf=True,       # Use sublinear scaling to reduce the impact of high term frequency
    min_df=20,               # Ignore terms that appear in fewer than 20 documents
    ngram_range=(1, 3),      # Consider unigrams, bigrams, and trigrams
    max_features=5000,       # Limit the number of features to 5000 to reduce dimensionality
    stop_words='english',    # Remove English stop words
    tokenizer=None,          # Use preprocessed text directly, no additional tokenization
    norm='l2'                # Apply L2 normalization to the feature vectors
)

# Transform the cleaned movie descriptions into a numerical feature matrix
features = tfidf.fit_transform(df['Description_clean'])
labels = df['Genre_id']


In [55]:
N = 5
unique_genres = df['Genre'].unique()

for genre in unique_genres:
    category_id = df[df['Genre'] == genre]['Genre_id'].iloc[0]
    features_chi2 = chi2(features, labels == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names_out())[indices]

    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    trigrams = [v for v in feature_names if len(v.split(' ')) == 3]

    print(f"\n==> {genre}:")
    print(f"  * Most Correlated Unigrams: {', '.join(unigrams[-N:])}")
    print(f"  * Most Correlated Bigrams: {', '.join(bigrams[-N:])}")
    print(f"  * Most Correlated Trigrams: {', '.join(trigrams[-N:])}")


==>  drama :
  * Most Correlated Unigrams: documentary, love, son, father, mother
  * Most Correlated Bigrams: run away, young woman, young man, year old, fall love
  * Most Correlated Trigrams: year old girl, change life forever, year old daughter, based true story, year old son

==>  thriller :
  * Most Correlated Unigrams: serial, suspect, murder, killer, thriller
  * Most Correlated Bigrams: thing start, young couple, dead body, dark secret, serial killer
  * Most Correlated Trigrams: film tell story, year old girl, feature length documentary, year old daughter, start new life

==>  adult :
  * Most Correlated Unigrams: gag, bondage, bound, sex, gagged
  * Most Correlated Bigrams: story woman, married couple, young lady, beautiful woman, bound gagged
  * Most Correlated Trigrams: feature length documentary, year old son, film tell story, year old boy, year old girl

==>  documentary :
  * Most Correlated Unigrams: explores, film, history, interview, documentary
  * Most Correlated

In [47]:
X = df['Description']
y = df['Genre_id']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=20,
                                                    stratify=y)

# Check of clases distribution in our samples
print("clases distribution in train sample:")
print(y_train.value_counts(normalize=True))
print("\nclases distribution in test sample:")
print(y_test.value_counts(normalize=True))
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, ngram_range=(1, 2), stop_words='english')

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF размеры:")
print(f"Обучающая выборка: {X_train_tfidf.shape}")
print(f"Тестовая выборка: {X_test_tfidf.shape}")

clases distribution in train sample:
Genre_id
8     0.251107
7     0.241564
5     0.137359
21    0.093581
13    0.040654
24    0.029341
0     0.024250
26    0.019036
18    0.016306
9     0.014461
2     0.014289
14    0.013478
19    0.012395
20    0.011928
1     0.010871
6     0.009321
3     0.009198
22    0.007969
23    0.007206
10    0.005952
16    0.005878
15    0.005116
4     0.004894
12    0.004476
11    0.003591
17    0.003345
25    0.002435
Name: proportion, dtype: float64

clases distribution in test sample:
Genre_id
8     0.251070
7     0.241552
5     0.137376
21    0.093552
13    0.040652
24    0.029364
0     0.024273
26    0.019035
18    0.016305
9     0.014461
2     0.014313
14    0.013502
19    0.012395
20    0.011952
1     0.010919
6     0.009296
3     0.009149
22    0.007968
23    0.007230
10    0.005976
16    0.005902
15    0.005091
4     0.004869
12    0.004501
11    0.003541
17    0.003320
25    0.002435
Name: proportion, dtype: float64
TF-IDF размеры:
Обучающая выборк

In [50]:
# Define the list of models to evaluate
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),  # Random Forest with limited depth
    LinearSVC(),  # Linear Support Vector Classifier
    MultinomialNB(),  # Multinomial Naive Bayes
    LogisticRegression(random_state=0, max_iter=1000),  # Logistic Regression with increased iterations
]

# Cross-validation parameters
CV = 5  # 5-Fold Cross-Validation
entries = []  # List to store cross-validation results

# Perform cross-validation for each model
for model in models:
    model_name = model.__class__.__name__  # Get the model's name
    accuracies = cross_val_score(model, X_train_tfidf, y_train, scoring='accuracy', cv=CV)  # Cross-validation
    for fold_idx, accuracy in enumerate(accuracies):  # Store results for each fold
        entries.append((model_name, fold_idx, accuracy))

# Create a DataFrame to store cross-validation results
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
metrics_entries = []

for model in models:
    model_name = model.__class__.__name__  # Get the model's name
    model.fit(X_train_tfidf, y_train)  # Train the model on the training set
    y_pred = model.predict(X_test_tfidf)  # Predict the labels for the test set

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)  # Accuracy
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)  # Weighted Precision
    recall = recall_score(y_test, y_pred, average='weighted')  # Weighted Recall
    f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted F1-Score

    # Add the calculated metrics to the list
    metrics_entries.append((model_name, accuracy, precision, recall, f1))

In [52]:
# Create a DataFrame to store the calculated metrics
metrics_df = pd.DataFrame(metrics_entries, columns=['model_name', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Combine cross-validation results with test set metrics
final_df = cv_df.groupby('model_name')['accuracy'].agg(['mean', 'std']).reset_index()  # Group by model name
final_df.rename(columns={'mean': 'Mean Accuracy', 'std': 'Standard Deviation'}, inplace=True)  # Rename columns
final_df = final_df.merge(metrics_df, on='model_name')  # Merge cross-validation and test set results
print("\nFinal Metrics Table:")
final_df



Финальная таблица метрик:


,model_name,Mean Accuracy,Standard Deviation,Accuracy,Precision,Recall,F1-Score
0,LinearSVC,0.583743,0.005081,0.585952,0.554010,0.585952,0.555854
1,LogisticRegression,0.574004,0.004678,0.584182,0.572996,0.584182,0.531366
2,MultinomialNB,0.460280,0.002996,0.468275,0.485833,0.468275,0.346402
3,RandomForestClassifier,0.412002,0.001475,0.415966,0.226292,0.415966,0.284920
